In [3]:
import numpy as np
import matplotlib.pyplot as plt

# Pre-Lab

In [1]:
def composite_trapezoid(f, a, b, N):
    if N < 2:
        raise ValueError("N must be at least 2.")
    h = (b - a) / (N - 1)
    x = [a + i * h for i in range(N)]
    return h * (0.5 * f(x[0]) + sum(f(xi) for xi in x[1:-1]) + 0.5 * f(x[-1]))


In [2]:
def composite_simpson(f, a, b, N):
    if N < 3 or N % 2 == 0:
        raise ValueError("N must be an odd number ≥ 3.")
    h = (b - a) / (N - 1)
    x = [a + i * h for i in range(N)]
    return (h / 3) * (
        f(x[0]) +
        4 * sum(f(x[i]) for i in range(1, N-1, 2)) +
        2 * sum(f(x[i]) for i in range(2, N-2, 2)) +
        f(x[-1])
    )


In [6]:
def gauss_quad(M, a, b, f):
    # Get the M Gauss-Legendre nodes and weights on [-1, 1]
    nodes, weights = np.polynomial.legendre.leggauss(M)
    
    # Map the nodes from [-1, 1] to [a, b]
    X = 0.5 * (b - a) * nodes + 0.5 * (a + b)
    
    # Transform weights accordingly; the scaling factor is (b-a)/2
    I = 0.5 * (b - a) * np.sum(weights * f(X))
    
    # For a single subinterval evaluation, we set nsplit = 1
    return I, X, 1


In [11]:

def adaptive_quad(a,b,f,tol,M,method):
  """
  Adaptive numerical integrator for \int_a^b f(x)dx
  
  Input:
  a,b - interval [a,b]
  f - function to integrate
  tol - absolute accuracy goal
  M - number of quadrature nodes per bisected interval
  method - function handle for integrating on subinterval
         - eg) eval_gauss_quad, eval_composite_simpsons etc.
  
  Output: I - the approximate integral
          X - final adapted grid nodes
          nsplit - number of interval splits
  """
  # 1/2^50 ~ 1e-15
  maxit = 50
  left_p = np.zeros((maxit,))
  right_p = np.zeros((maxit,))
  s = np.zeros((maxit,1))
  left_p[0] = a; right_p[0] = b;
  # initial approx and grid
  s[0],x,_ = method(M,a,b,f);
  # save grid
  X = []
  X.append(x)
  j = 1;
  I = 0;
  nsplit = 1;
  while j < maxit:
    # get midpoint to split interval into left and right
    c = 0.5*(left_p[j-1]+right_p[j-1]);
    # compute integral on left and right spilt intervals
    s1,x,_ = method(M,left_p[j-1],c,f); X.append(x)
    s2,x,_ = method(M,c,right_p[j-1],f); X.append(x)
    if np.max(np.abs(s1+s2-s[j-1])) > tol:
      left_p[j] = left_p[j-1]
      right_p[j] = 0.5*(left_p[j-1]+right_p[j-1])
      s[j] = s1
      left_p[j-1] = 0.5*(left_p[j-1]+right_p[j-1])
      s[j-1] = s2
      j = j+1
      nsplit = nsplit+1
    else:
      I = I+s1+s2
      j = j-1
      if j == 0:
        j = maxit
  return I,np.unique(X),nsplit

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
/var/folders/bk/w3kf9k496lv9ntkhhfbltxs80000gn/T/ipykernel_18114/605815979.py:2: SyntaxWarning: invalid escape sequence '\i'
  """


In [9]:
def adaptive_gauss_quad(a, b, f, tol, M, method=gauss_quad):

    return adaptive_quad(a, b, f, tol, M, method)


def adaptive_composite_trap(a, b, f, tol, M, method=composite_trapezoid):
    """
    Adaptive quadrature using the Composite Trapezoidal Rule as the underlying method.
    """
    return adaptive_quad(a, b, f, tol, M, method)


def adaptive_composite_simpsons(a, b, f, tol, M, method=composite_simpson):
    """
    Adaptive quadrature using the Composite Simpson's Rule as the underlying method.
    """
    return adaptive_quad(a, b, f, tol, M, method)

In [12]:

# Example function to integrate and its true value (modify as desired):
f = lambda x: np.sin(1/x)
I_true = 1.14555
a, b = 0.1, 2.0

tol = 1e-14
M = 5  

# Choose the adaptive routine you want to test:
# method_func = adaptive_gauss_quad
# method_func = adaptive_composite_trap
method_func = adaptive_composite_simpsons  # try this one

I_adapt, mesh, nsplit = method_func(a, b, f, tol, M)

print("Adaptive Quadrature Result:", I_adapt)
print("True Value:", I_true)
print("Relative Error:", np.abs(I_adapt - I_true)/I_true)
print("Total subinterval evaluations:", nsplit)

# Optionally, plot the mesh used:
plt.figure()
plt.plot(mesh, f(mesh), 'o-')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('Adaptive Quadrature Mesh')
plt.show()

/var/folders/bk/w3kf9k496lv9ntkhhfbltxs80000gn/T/ipykernel_18114/605815979.py:2: SyntaxWarning: invalid escape sequence '\i'
  """


TypeError: '<' not supported between instances of 'function' and 'int'